In [ ]:
# import
import sys
sys.path.append('../')

import logging
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt

from tissue_clustering import Tissues, find_DBSCAN_clusters
from loss import loss_fct
from visualization_utils import show_clusters, show_only_clusters
from slicing import slice_volume

In [ ]:
# Load the image data
volume = sitk.ReadImage('..\MRI\Labels\labels_00001.nii')
img_array = sitk.GetArrayFromImage(volume)
print("volume shape: ", volume.GetSize())
print("img_array shape: ", img_array.shape)

# Take a 2D image slice
slice_num = 670
slice = img_array[:, slice_num, :]
plt.imshow(slice, origin='lower', aspect=6)
plt.show()

In [ ]:
tissues = { "bones": 1,
            "tendins": 2,
            "ulnar": 3,}
tissuesClass = Tissues(tissues_dict=tissues)

In [ ]:
print("origin of volume: ", volume.GetOrigin())
print("world coordinates of index: ", volume.TransformContinuousIndexToPhysicalPoint((0, 670, 0)))

In [ ]:
from matplotlib.animation import FuncAnimation

t = [160, 155, 150, 148, 146, 142,  140, 140, 115, 120, 125, 125, 130, 130, 135, 138, 140, 140,  140]
z = [  0,  -5,   0,   0,   5,  15, 19.3, -10,   0,   0,   0,   5,   -8,  8,   0, -10, -10,  10, 19.3]
o = volume.GetOrigin()

# Sample functions for demonstration
def linear_function(x, m, b):
  return m * x + b

# Create a figure and a gridspec with two rows and two columns
fig = plt.figure(constrained_layout=True, figsize=(8, 6))
gs = fig.add_gridspec(2, 2)

# Add subplots
ax1 = fig.add_subplot(gs[:, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, 1])

# Subplot 1: Image with dashed line
ax1.imshow(img_array[40,:,:])
x_dash = np.arange(img_array.shape[2])
b = volume.TransformPhysicalPointToIndex([o[0], o[1]+t[0], o[2]])[1]
y_dash = linear_function(x_dash, np.tan(np.deg2rad(z[0])), b)
ax1.set_title(f"Section {0}")
line = ax1.plot(x_dash, y_dash, linestyle="--", color="red")[0]
ax1.set_title('Slice cut')

# ACTION
slice = slice_volume(z_rotation=0., x_rotation=0., translation=[0, 165, 0], volume=volume)
slice = sitk.GetArrayFromImage(slice)[:, 0, :]
ax2.imshow(slice, origin='lower', aspect=6)
ax2.set_title(f'Slice {0}')

# OBSERVATION
clusters = {}
clusters['bones'] = find_DBSCAN_clusters(tissues['bones'], slice, eps=4.1, min_samples=46)
clusters['tendins'] = find_DBSCAN_clusters(tissues['tendins'], slice, eps=4.1, min_samples=46)
clusters['ulnar'] = find_DBSCAN_clusters(tissues['ulnar'], slice, eps=2.5, min_samples=18)
ax3.set_title('Clusters')
ax3 = show_clusters(clusters, slice)
# REWARD
loss = loss_fct(clusters)
plt.text(0, 0, f"Loss: {loss:.2f}", fontsize=12, color='red')

plt.show()
